In [ ]:
!pip install praat-textgrids
!pip install jiwer

In [ ]:
import sys
from absl import flags
FLAGS = flags.FLAGS
flags.DEFINE_string('normalizers_file', '/kaggle/input/normalizers/normalizers.pkl', 'file with pickled feature normalizers')

flags.DEFINE_list('remove_channels', [], 'channels to remove')
flags.DEFINE_list('silent_data_directories', ['/kaggle/input/emgdata/emg_data/silent_parallel_data'], 'silent data locations')
flags.DEFINE_list('voiced_data_directories', ['/kaggle/input/emgdata/emg_data/voiced_parallel_data','/kaggle/input/emgdata/emg_data/nonparallel_data'], 'voiced data locations')
flags.DEFINE_string('testset_file', '/kaggle/input/testesetlarge/testset_largedev.json', 'file with testset indices')
flags.DEFINE_string('text_align_directory', '/kaggle/input/textaligns/text_alignments', 'directory with alignment files')

flags.DEFINE_boolean('debug', False, 'debug')
flags.DEFINE_string('output_directory', '/kaggle/working/outputs', 'where to save models and outputs')
flags.DEFINE_integer('batch_size', 16, 'training batch size')
flags.DEFINE_float('learning_rate', 3e-4, 'learning rate')
flags.DEFINE_integer('learning_rate_warmup', 1000, 'steps of linear warmup')
flags.DEFINE_integer('learning_rate_patience', 5, 'learning rate decay patience')
flags.DEFINE_string('start_training_from', None, 'start training from this model')
flags.DEFINE_float('l2', 0, 'weight decay')
flags.DEFINE_string('evaluate_saved', None, 'run evaluation on given model file')
flags.DEFINE_integer('accumulation_steps', 4, 'número debatches para acumular gradientes')

FLAGS(sys.argv[1:])


In [ ]:
%%writefile data_utils.py

import string

import numpy as np
import librosa
import soundfile as sf
from textgrids import TextGrid
import jiwer
from unidecode import unidecode

import torch
import matplotlib.pyplot as plt

from absl import flags
FLAGS = flags.FLAGS

phoneme_inventory = ['aa','ae','ah','ao','aw','ax','axr','ay','b','ch','d','dh','dx','eh','el','em','en','er','ey','f','g','hh','hv','ih','iy','jh','k','l','m','n','nx','ng','ow','oy','p','r','s','sh','t','th','uh','uw','v','w','y','z','zh','sil']

def normalize_volume(audio): #recebe um sinal de áudio e realiza uma normalização de volume nele.
    rms = librosa.feature.rms(audio)  #calcula o valor RMS (root mean square) do sinal de áudio 
    max_rms = rms.max() + 0.01
    target_rms = 0.2
    audio = audio * (target_rms/max_rms) #normaliza o sinal de áudio multiplicando-o por um fator que ajusta o RMS para um valor de destino (0.2)
    max_val = np.abs(audio).max()
    if max_val > 1.0: # this shouldn't happen too often with the target_rms of 0.2
        audio = audio / max_val  #Se o valor máximo absoluto do sinal de áudio for maior que 1.0, o sinal é dividido pelo valor máximo para evitar a saturação.
    return audio

def dynamic_range_compression_torch(x, C=1, clip_val=1e-5):  #realiza uma compressão de faixa dinâmica em um tensor
    return torch.log(torch.clamp(x, min=clip_val) * C)

def spectral_normalize_torch(magnitudes):  #realiza a normalização espectral
    output = dynamic_range_compression_torch(magnitudes)
    return output

mel_basis = {}
hann_window = {}

def mel_spectrogram(y, n_fft, num_mels, sampling_rate, hop_size, win_size, fmin, fmax, center=False):  # calcula o espectrograma mel de um sinal de áudio
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))

    global mel_basis, hann_window
    if fmax not in mel_basis: #verificam se já foi calculada a base de mel correspondente à frequência máxima fmax. Se não tiver sido calculada, a função librosa.filters.mel 
                                #é usada para calcular a base de mel com os parâmetros fornecidos
        mel = librosa.filters.mel(sr=sampling_rate, n_fft=n_fft, n_mels=num_mels, fmin=fmin, fmax=fmax)

        #print('Mel ', mel)
        mel_basis[str(fmax)+'_'+str(y.device)] = torch.from_numpy(mel).float().to(y.device)
        hann_window[str(y.device)] = torch.hann_window(win_size).to(y.device)
        #print('Mel_basis ', mel_basis)

    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect') #preenche o sinal de áudio y com reflexão antes do cálculo do espectrograma. 
                                                                                #O sinal é expandido com uma dimensão extra e é aplicado um preenchimento refletivo em ambas as extremidades.
    y = y.squeeze(1) #a dimensão extra é removida.
    #print('y ', y)
    #spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=hann_window[str(y.device)], #calcula a transformada de Fourier de curto tempo (STFT) do sinal de áudio y
                      #center=center, pad_mode='reflect', normalized=False, onesided=True)  #O espectrograma é calculado apenas para a metade positiva das frequências (onesided=True).
    #print ('Resutado do stft ', spec)
    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=hann_window[str(y.device)],
                  center=center, pad_mode='reflect', normalized=False, onesided=True, return_complex=False)
    #print ('Resutado do stft ', spec)
    spec = torch.sqrt(spec.pow(2).sum(-1)+(1e-9)) #Essa linha calcula o módulo (magnitude) do espectrograma. O espectrograma é elevado ao quadrado, somado ao valor de 1e-9 para evitar divisão por zero
    # Imprimir as dimensões de mel_basis
    #print ('Resutado do sqrt ', spec)
    #print(len(mel_basis))
    #for key, value in mel_basis.items():
        #print(f"Dimensões de {key}: {value.shape}")
    #print('Dimensões de spec ', spec.shape)
    spec = torch.matmul(mel_basis[str(fmax)+'_'+str(y.device)], spec)

    
    #spec = mel_basis[str(fmax)+'_'+str(y.device)] @ spec

    #print ('Resutado da multiplicação ', spec)
    #print('Dimensões de spec ', spec.shape)
    spec = spectral_normalize_torch(spec) #O resultado é normalizado espectralmente

    return spec

def load_audio(filename, start=None, end=None, max_frames=None, renormalize_volume=False): #carrega um arquivo de áudio, realiza pré-processamento nele e retorna o espectrograma mel correspondente
    audio, r = sf.read(filename)

    if len(audio.shape) > 1:
        audio = audio[:,0] # verifica se o sinal de áudio possui mais de uma dimensão, o que indicaria que é um áudio estéreo. Nesse caso, seleciona apenas o primeiro canal de áudio.
    if start is not None or end is not None:
        audio = audio[start:end] #permitem selecionar uma parte específica do sinal de áudio

    if renormalize_volume:
        audio = normalize_volume(audio)
    if r == 16000:
        audio = librosa.resample(audio, orig_sr=16000, target_sr=22050)  #verificam a taxa de amostragem r do áudio. Se for igual a 16000, o áudio é ressampleado para a taxa de amostragem de 22050 Hz usando a função librosa.resample. 
    else:
        assert r == 22050
    audio = np.clip(audio, -1, 1) # realiza um ajuste no intervalo de valores do sinal de áudio. Por vezes, o ressampleamento pode fazer com que alguns valores ultrapassem o intervalo [-1, 1], 
                                    #então essa linha garante que todos os valores estejam dentro desse intervalo.
    pytorch_mspec = mel_spectrogram(torch.tensor(audio, dtype=torch.float32).unsqueeze(0), 1024, 80, 22050, 256, 1024, 0, 8000, center=False)  #calculam o espectrograma mel do sinal de áudio
    mspec = pytorch_mspec.squeeze(0).T.numpy() #O resultado é convertido em uma matriz numpy e atribuído a mspec.
    if max_frames is not None and mspec.shape[0] > max_frames:  #verifica se o parâmetro max_frames foi fornecido e se o número de quadros (linhas) do espectrograma excede max_frames
        mspec = mspec[:max_frames,:] #o espectrograma é recortado para ter no máximo max_frames quadros.
    return mspec

def double_average(x):  #suaviza um sinal x aplicando uma média móvel dupla, ou seja, realizando duas convoluções consecutivas com um filtro médio. Isso ajuda a reduzir o ruído e suavizar o sinal.
    assert len(x.shape) == 1  # verificam se o sinal x tem apenas uma dimensão. Caso contrário, é lançado um erro.
    f = np.ones(9)/9.0  #cria uma matriz de tamanho 9 preenchida com o valor 1.0 e, em seguida, divide todos os elementos por 9.0. Essa matriz f será usada como um filtro médio.
    v = np.convolve(x, f, mode='same')  #aplica a convolução entre o sinal x e o filtro f. A opção mode='same' garante que o tamanho da saída seja o mesmo que o tamanho de x
    w = np.convolve(v, f, mode='same')  #aplica a convolução entre o sinal v e o filtro f. A opção mode='same' garante que o tamanho da saída seja o mesmo que o tamanho de x
    return w

def get_emg_features(emg_data, debug=False): #calcula recursos relacionados ao sinal de eletromiografia (EMG) a partir dos dados de EMG fornecido. Esses recursos incluem médias de janelas, valores RMS, 
                                    #taxa de cruzamento por zero e espectrograma de curto prazo. Esses recursos podem ser usados posteriormente para análise e processamento adicional dos sinais de EMG.
    xs = emg_data - emg_data.mean(axis=0, keepdims=True)  #calculam xs, que é o sinal de EMG centrado em torno da média. 
                                                            #É subtraída a média de cada coluna dos dados de EMG (emg_data) usando a função mean ao longo do eixo 0.
    frame_features = []
    for i in range(emg_data.shape[1]):
        x = xs[:,i]  # x: coluna atual de xs
        w = double_average(x) # w: sinal resultante da aplicação da função double_average em x, ou seja, o sinal suavizado
        p = x - w # p: sinal resultante da subtração de w de x, representando as partes pulsativas do sinal
        r = np.abs(p) #r: sinal resultante do valor absoluto de p, representando a magnitude das partes pulsativas do sinal

        w_h = librosa.util.frame(w, frame_length=16, hop_length=6).mean(axis=0) # w_h: média das janelas de 16 amostras de w com um deslocamento de 6 amostras
        p_w = librosa.feature.rms(y=w, frame_length=16, hop_length=6, center=False)  #p_w: valor RMS (Root Mean Square) das janelas de 16 amostras de w com um deslocamento de 6 amostras
        p_w = np.squeeze(p_w, 0)
        p_r = librosa.feature.rms(y=r, frame_length=16, hop_length=6, center=False)  #p_r: valor RMS das janelas de 16 amostras de r com um deslocamento de 6 amostras
        p_r = np.squeeze(p_r, 0)
        z_p = librosa.feature.zero_crossing_rate(p, frame_length=16, hop_length=6, center=False) #z_p: taxa de cruzamento por zero das janelas de 16 amostras de p com um deslocamento de 6 amostras
        z_p = np.squeeze(z_p, 0)
        r_h = librosa.util.frame(r, frame_length=16, hop_length=6).mean(axis=0) #r_h: média das janelas de 16 amostras de r com um deslocamento de 6 amostras

        s = abs(librosa.stft(np.ascontiguousarray(x), n_fft=16, hop_length=6, center=False))  #calcula o espectrograma de curto prazo do sinal x usando a Transformada de Fourier de Curto Prazo (STFT)
        # s has feature dimension first and time second

        if debug:
            plt.subplot(7,1,1)
            plt.plot(x)
            plt.subplot(7,1,2)
            plt.plot(w_h)
            plt.subplot(7,1,3)
            plt.plot(p_w)
            plt.subplot(7,1,4)
            plt.plot(p_r)
            plt.subplot(7,1,5)
            plt.plot(z_p)
            plt.subplot(7,1,6)
            plt.plot(r_h)

            plt.subplot(7,1,7)
            plt.imshow(s, origin='lower', aspect='auto', interpolation='nearest')

            plt.show()

        frame_features.append(np.stack([w_h, p_w, p_r, z_p, r_h], axis=1))  #empilham os recursos calculados para cada coluna em uma lista. Os recursossão empilhados verticalmente, resultando em uma matriz 2D.
        frame_features.append(s.T) #o espectrograma s é transposto e adicionado à lista frame_features.

    frame_features = np.concatenate(frame_features, axis=1) #concatena todos os elementos da lista frame_features ao longo do eixo 1, resultando em uma matriz unidimensional final
    return frame_features.astype(np.float32)

class FeatureNormalizer(object):  #implementa um normalizador de recursos.Ela fornece métodos para normalizar uma amostra de recurso e desfazer a normalização, 
                #aplicando as médias e os desvios padrão calculados. Esse normalizador pode ser útil para preparar os dados antes de usá-los em um modelo de aprendizado de máquina.
    def __init__(self, feature_samples, share_scale=False):#é o construtor da classe. Ele recebe uma lista de amostras de recursos (feature_samples), que são matrizes 2D com dimensões (tempo, recurso). 
        """ features_samples should be list of 2d matrices with dimension (time, feature) """
        feature_samples = np.concatenate(feature_samples, axis=0)  #as amostras de recursos são concatenadas ao longo do eixo 0 para formar uma única matriz
        self.feature_means = feature_samples.mean(axis=0, keepdims=True)  #as médias dos recursos são calculadas ao longo do eixo 0 e armazenadas em self.feature_means
        if share_scale:
            self.feature_stddevs = feature_samples.std()  #Se share_scale for True, o desvio padrão de todos os recursos é calculado e armazenado em self.feature_stddevs
        else:
            self.feature_stddevs = feature_samples.std(axis=0, keepdims=True) #Caso contrário, os desvios padrão de cada recurso são calculados separadamente e armazenados em self.feature_stddevs.

    def normalize(self, sample): #recebe uma amostra de recurso (sample) e normaliza essa amostra subtraindo as médias dos recursos (self.feature_means) e dividindo pelo desvio padrão dos recursos
        sample -= self.feature_means
        sample /= self.feature_stddevs
        return sample

    def inverse(self, sample): #ecebe uma amostra de recurso normalizada e realiza a operação inversa da normalização. Primeiro, a amostra é multiplicada pelo desvio padrão dos recursos. 
                                #Em seguida, a média dos recursos (self.feature_means) é adicionada de volta à amostra. 
        sample = sample * self.feature_stddevs
        sample = sample + self.feature_means
        return sample

def combine_fixed_length(tensor_list, length): #combina uma lista de tensores em um único tensor de comprimento fixo. 
                                #Ele garante que os dados sejam combinados em um tensor de tamanho fixo, preenchendo com zeros, se necessário.
    total_length = sum(t.size(0) for t in tensor_list) #calcula o comprimento total somando o tamanho (dimensão 0) de cada tensor na lista.
    if total_length % length != 0:#verifica se o comprimento total não é divisível pelo comprimento desejado
        pad_length = length - (total_length % length)  #Se não for, calcula o comprimento de preenchimento necessário para tornar o total divisível pelo comprimento desejado
        tensor_list = list(tensor_list) # copy
        tensor_list.append(torch.zeros(pad_length,*tensor_list[0].size()[1:], dtype=tensor_list[0].dtype, device=tensor_list[0].device)) #um tensor preenchido com zeros é criado com o comprimento de 
                                                                #preenchimento necessário e as mesmas dimensões do primeiro tensor na lista. Esse tensor de preenchimento é anexado à lista de tensores.
        total_length += pad_length 
    tensor = torch.cat(tensor_list, 0)  #os tensores na lista (incluindo o tensor de preenchimento, se adicionado) são concatenados ao longo da dimensão 0 para formar um único tensor.
    n = total_length // length #o número de segmentos de comprimento fixo que podem ser extraídos do tensor é calculado dividindo o comprimento total pelo comprimento desejado.
    return tensor.view(n, length, *tensor.size()[1:])  #o tensor é redimensionado para ter as dimensões (n, length, ...), onde n é o número de segmentos e ... representa as dimensões restantes dos tensores originais.

def decollate_tensor(tensor, lengths):  #desagrega um tensor em uma lista de tensores com comprimentos diferentes. Essa função é útil quando você deseja separar um tensor em segmentos de comprimentos diferentes, 
                                    #conforme especificado por uma lista de comprimentos. Pode ser usado, por exemplo, para processar lotes de dados em tamanhos diferentes após a etapa de inferência em um modelo.
    b, s, d = tensor.size()  # obtem as dimensões do tensor original. b representa o tamanho do lote, s representa o comprimento dos segmentos e d representa a dimensão dos recursos.
    tensor = tensor.view(b*s, d) # o tensor é redimensionado usando .view() para ter uma forma de (b * s, d). Isso combina o tamanho do lote com o comprimento dos segmentos.
    results = []
    idx = 0
    for length in lengths: #Para cada comprimento, é verificado se a posição atual mais o comprimento está dentro dos limites do tensor. Se não estiver, um erro de assert é acionado.
        assert idx + length <= b * s
        results.append(tensor[idx:idx+length]) #o segmento correspondente é extraído do tensor, começando na posição idx e com o comprimento especificado. O segmento é adicionado à lista results.
        idx += length
    return results

def splice_audio(chunks, overlap): #combina várias partes de áudio sobrepostas em um único áudio.Essa função é útil para combinar partes de áudio sobrepostas, como segmentos de áudio em uma gravação 
                                        #contínua, onde a sobreposição ajuda a suavizar a transição entre as partes.
    chunks = [c.copy() for c in chunks] # copy so we can modify in place

    assert np.all([c.shape[0]>=overlap for c in chunks]) #é verificado se todas as partes de áudio têm um tamanho maior ou igual à sobreposição especificada. Isso é importante para garantir 
                                                            #que haja dados suficientes para aplicar a sobreposição corretamente.

    result_len = sum(c.shape[0] for c in chunks) - overlap*(len(chunks)-1) #soma os tamanhos de todas as partes de áudio e subtraindo o tamanho da sobreposição entre as partes para o comprimento total
    result = np.zeros(result_len, dtype=chunks[0].dtype)  #Um array de zeros chamado result é inicializado com o comprimento total calculado e o tipo de dados da primeira parte de áudio.

    ramp_up = np.linspace(0,1,overlap) #Duas rampas, ramp_up e ramp_down, são criadas usando np.linspace() para representar as funções de aumento e diminuição gradual da amplitude durante a sobreposição.
    ramp_down = np.linspace(1,0,overlap)

    i = 0
    for chunk in chunks:
        l = chunk.shape[0]  #Em um loop for, cada parte de áudio é processada individualmente. Para cada parte de áudio, seu comprimento l é obtido.

        # note: this will also fade the beginning and end of the result
        chunk[:overlap] *= ramp_up  #As partes de áudio são multiplicadas pelos valores correspondentes nas rampas ramp_up e ramp_down. Isso aplica a sobreposição gradual no início e no final de cada parte de áudio.
        chunk[-overlap:] *= ramp_down

        result[i:i+l] += chunk #A parte de áudio processada é adicionada ao resultado final a partir da posição i. A variável i é atualizada para a próxima posição correta no resultado, levando em 
                                    #consideração o tamanho da parte de áudio e a sobreposição.
        i += l-overlap

    return result

def print_confusion(confusion_mat, n=10): #imprime informações sobre as confusões mais comuns em uma matriz de confusão. Essa função é útil para analisar e visualizar as confusões mais comuns em uma matriz de 
                            #confusão, fornecendo insights sobre o desempenho do modelo de classificação em relação a classes específicas.
    # axes are (pred, target)
    target_counts = confusion_mat.sum(0) + 1e-4  #calcula o número de ocorrências de cada classe alvo na matriz de confusão. Isso é feito somando os valores de cada coluna da matriz e adicionando um 
                                    #pequeno valor (1e-4) para evitar divisão por zero.
    aslist = []
    for p1 in range(len(phoneme_inventory)): #a função itera sobre todas as combinações únicas de classes alvo (p1 e p2). As confusões são calculadas somando as ocorrências nas células 
                                    #correspondentes na matriz de confusão e dividindo pelo número total de ocorrências das duas classes alvo.
        for p2 in range(p1):
            if p1 != p2:
                aslist.append(((confusion_mat[p1,p2]+confusion_mat[p2,p1])/(target_counts[p1]+target_counts[p2]), p1, p2)) #As confusões são armazenadas em uma lista aslist como uma tupla contendo a taxa de confusão,
                                                                                                                #o índice p1 da classe alvo, e o índice p2 da classe alvo.
    aslist.sort() #classifica em ordem crescente com base na taxa de confusão
    aslist = aslist[-n:]  #é selecionado o top n das confusões mais comuns.
    max_val = aslist[-1][0]  #O valor máximo e o valor mínimo de confusão são obtidos a partir da lista aslist.
    min_val = aslist[0][0]
    val_range = max_val - min_val
    print('Common confusions (confusion, accuracy)') 
    for v, p1, p2 in aslist:
        p1s = phoneme_inventory[p1]
        p2s = phoneme_inventory[p2]
        print(f'{p1s} {p2s} {v*100:.1f} {(confusion_mat[p1,p1]+confusion_mat[p2,p2])/(target_counts[p1]+target_counts[p2])*100:.1f}')
        #^ imprime as informações sobre as confusões mais comuns. Ela exibe a classe alvo p1, a classe alvo p2, a taxa de confusão (multiplicada por 100 para exibição em porcentagem) e a precisão (também 
        # multiplicada por 100).

def read_phonemes(textgrid_fname, max_len=None): #lê os fonemas de um arquivo TextGrid e os converte em uma sequência de índices de fonemas, usada para treinar e avaliar modelos de processamento de linguagem.
    tg = TextGrid(textgrid_fname)
    phone_ids = np.zeros(int(tg['phones'][-1].xmax*86.133)+1, dtype=np.int64)  #cria um array phone_ids preenchido com valores -1. O tamanho do array é calculado com base no tempo máximo encontrado 
            #no arquivo TextGrid multiplicado por 86.133. O valor 86.133 é usado para converter o tempo do TextGrid para a taxa de amostragem de 22050 Hz, que é a taxa de amostragem mencionada anteriormente.
    phone_ids[:] = -1  
    phone_ids[-1] = phoneme_inventory.index('sil') #o último valor do array é definido como o índice do fonema 'sil' no inventário de fonemas. Isso garante que a lista seja longa o suficiente para cobrir 
                                                    #todo o comprimento da sequência original.
    for interval in tg['phones']: #percorre cada intervalo de fonema no arquivo TextGrid e mapeia o fonema correspondente para o seu índice no inventário de fonemas.
        phone = interval.text.lower() 
        if phone in ['', 'sp', 'spn']: # Se o fonema for uma string vazia, 'sp' ou 'spn', ele é substituído por 'sil'
            phone = 'sil'
        if phone[-1] in string.digits: #Se o fonema tiver um dígito no final, o dígito é removido.
            phone = phone[:-1]
        ph_id = phoneme_inventory.index(phone) #Os índices de fonemas são atribuídos aos intervalos de tempo correspondentes no array phone_ids.
        phone_ids[int(interval.xmin*86.133):int(interval.xmax*86.133)] = ph_id
    assert (phone_ids >= 0).all(), 'missing aligned phones' #verifica se todos os valores de phone_ids são não negativos (ou seja, todos os fonemas foram encontrados e mapeados corretamente). 
                                #Caso contrário, uma exceção é lançada indicando que há fonemas ausentes na sequência alinhada.

    if max_len is not None:  #Se max_len for especificado, a sequência de fonemas é truncada para o comprimento máximo especificado
        phone_ids = phone_ids[:max_len]
        assert phone_ids.shape[0] == max_len #A função verifica se o comprimento da sequência truncada é igual a max_len.
    return phone_ids

class TextTransform(object): #implementa transformações de texto úteis, como limpeza de texto, mapeamento de texto para sequências de números inteiros e mapeamento inverso de sequências de números inteiros para texto.
    def __init__(self):
        self.transformation = jiwer.Compose([jiwer.RemovePunctuation(), jiwer.ToLowerCase()])  #transformation é um objeto jiwer.Compose que encapsula uma série de transformações de texto, como remoção de pontuação 
                                #e conversão para minúsculas. Essas transformações são realizadas pelo pacote jiwer.
        self.chars = string.ascii_lowercase+string.digits+' ' #chars é uma string que contém todos os caracteres permitidos no texto, incluindo letras minúsculas, dígitos e espaço em branco.

    def clean_text(self, text): #Limpa o texto aplicando as transformações definidas.
        text = unidecode(text)  #remove quaisquer caracteres acentuados ou diacríticos
        text = self.transformation(text) #as transformações definidas em self.transformation são aplicadas
        return text

    def text_to_int(self, text): #Converte o texto em uma sequência de números inteiros
        text = self.clean_text(text) #O texto é limpo usando o método clean_text
        return [self.chars.index(c) for c in text] #cada caractere do texto limpo é mapeado para o seu índice correspondente em self.chars

    def int_to_text(self, ints):  #Converte uma sequência de números inteiros em texto
        return ''.join(self.chars[i] for i in ints)  #Cada número inteiro em ints é mapeado para o caractere correspondente em self.chars. 

In [ ]:
%%writefile read_emg.py

import re
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import defaultdict
import scipy
import json
import copy
import sys
import pickle
import string
import logging
from functools import lru_cache
from copy import copy



import librosa
import soundfile as sf

import torch

from data_utils import load_audio, get_emg_features, FeatureNormalizer, phoneme_inventory, read_phonemes, TextTransform

from scipy import signal

from absl import flags
FLAGS = flags.FLAGS

def remove_drift(signal, fs):
    b, a = scipy.signal.butter(3, 2, 'highpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

def notch(signal, freq, sample_frequency):
    b, a = scipy.signal.iirnotch(freq, 30, sample_frequency)
    return scipy.signal.filtfilt(b, a, signal)

def notch_harmonics(signal, freq, sample_frequency):
    for harmonic in range(1,8):
        signal = notch(signal, freq*harmonic, sample_frequency)
    return signal

def subsample(signal, new_freq, old_freq):
    times = np.arange(len(signal))/old_freq
    sample_times = np.arange(0, times[-1], 1/new_freq)
    result = np.interp(sample_times, times, signal)
    return result

def apply_to_all(function, signal_array, *args, **kwargs):
    results = []
    for i in range(signal_array.shape[1]):
        results.append(function(signal_array[:,i], *args, **kwargs))
    return np.stack(results, 1)

def load_utterance(base_dir, index, limit_length=False, debug=False, text_align_directory=None):
    index = int(index)
    raw_emg = np.load(os.path.join(base_dir, f'{index}_emg.npy'))
    before = os.path.join(base_dir, f'{index-1}_emg.npy')
    after = os.path.join(base_dir, f'{index+1}_emg.npy')
    if os.path.exists(before):
        raw_emg_before = np.load(before)
    else:
        raw_emg_before = np.zeros([0,raw_emg.shape[1]])
    if os.path.exists(after):
        raw_emg_after = np.load(after)
    else:
        raw_emg_after = np.zeros([0,raw_emg.shape[1]])

    x = np.concatenate([raw_emg_before, raw_emg, raw_emg_after], 0)
    x = apply_to_all(notch_harmonics, x, 60, 1000)
    x = apply_to_all(remove_drift, x, 1000)
    x = x[raw_emg_before.shape[0]:x.shape[0]-raw_emg_after.shape[0],:]
    emg_orig = apply_to_all(subsample, x, 689.06, 1000)
    x = apply_to_all(subsample, x, 516.79, 1000)
    emg = x

    for c in FLAGS.remove_channels:
        emg[:,int(c)] = 0
        emg_orig[:,int(c)] = 0

    emg_features = get_emg_features(emg)

    mfccs = load_audio(os.path.join(base_dir, f'{index}_audio_clean.flac'),
            max_frames=min(emg_features.shape[0], 800 if limit_length else float('inf')))

    if emg_features.shape[0] > mfccs.shape[0]:
        emg_features = emg_features[:mfccs.shape[0],:]
    assert emg_features.shape[0] == mfccs.shape[0]
    emg = emg[6:6+6*emg_features.shape[0],:]
    emg_orig = emg_orig[8:8+8*emg_features.shape[0],:]
    assert emg.shape[0] == emg_features.shape[0]*6

    with open(os.path.join(base_dir, f'{index}_info.json')) as f:
        info = json.load(f)

    sess = os.path.basename(base_dir)
    tg_fname = f'{text_align_directory}/{sess}/{sess}_{index}_audio.TextGrid'
    if os.path.exists(tg_fname):
        phonemes = read_phonemes(tg_fname, mfccs.shape[0])
    else:
        phonemes = np.zeros(mfccs.shape[0], dtype=np.int64)+phoneme_inventory.index('sil')

    return mfccs, emg_features, info['text'], (info['book'],info['sentence_index']), phonemes, emg_orig.astype(np.float32)

class EMGDirectory(object):
    def __init__(self, session_index, directory, silent, exclude_from_testset=False):
        self.session_index = session_index
        self.directory = directory
        self.silent = silent
        self.exclude_from_testset = exclude_from_testset

    def __lt__(self, other):
        return self.session_index < other.session_index

    def __repr__(self):
        return self.directory

class SizeAwareSampler(torch.utils.data.Sampler):
    def __init__(self, emg_dataset, max_len):
        self.dataset = emg_dataset
        self.max_len = max_len

    def __iter__(self):
        indices = list(range(len(self.dataset)))
        random.shuffle(indices)
        batch = []
        batch_length = 0
        for idx in indices:
            directory_info, file_idx = self.dataset.example_indices[idx]
            with open(os.path.join(directory_info.directory, f'{file_idx}_info.json')) as f:
                info = json.load(f)
            if not np.any([l in string.ascii_letters for l in info['text']]):
                continue
            length = sum([emg_len for emg_len, _, _ in info['chunks']])
            if length > self.max_len:
                logging.warning(f'Warning: example {idx} cannot fit within desired batch length')
            if length + batch_length > self.max_len:
                yield batch
                batch = []
                batch_length = 0
            batch.append(idx)
            batch_length += length
        # dropping last incomplete batch

class EMGDataset(torch.utils.data.Dataset):
    def __init__(self, base_dir=None, limit_length=False, dev=False, test=False, no_testset=False, no_normalizers=False):

        self.text_align_directory = FLAGS.text_align_directory

        if no_testset:
            devset = []
            testset = []
        else:
            with open(FLAGS.testset_file) as f:
                testset_json = json.load(f)
                devset = testset_json['dev']
                testset = testset_json['test']

        directories = []
        if base_dir is not None:
            directories.append(EMGDirectory(0, base_dir, False))
        else:
            for sd in FLAGS.silent_data_directories:
                for session_dir in sorted(os.listdir(sd)):
                    directories.append(EMGDirectory(len(directories), os.path.join(sd, session_dir), True))

            has_silent = len(FLAGS.silent_data_directories) > 0
            for vd in FLAGS.voiced_data_directories:
                for session_dir in sorted(os.listdir(vd)):
                    directories.append(EMGDirectory(len(directories), os.path.join(vd, session_dir), False, exclude_from_testset=has_silent))

        self.example_indices = []
        self.voiced_data_locations = {} # map from book/sentence_index to directory_info/index
        for directory_info in directories:
            for fname in os.listdir(directory_info.directory):
                m = re.match(r'(\d+)_info.json', fname)
                if m is not None:
                    idx_str = m.group(1)
                    with open(os.path.join(directory_info.directory, fname)) as f:
                        info = json.load(f)
                        if info['sentence_index'] >= 0: # boundary clips of silence are marked -1
                            location_in_testset = [info['book'], info['sentence_index']] in testset
                            location_in_devset = [info['book'], info['sentence_index']] in devset
                            if (test and location_in_testset and not directory_info.exclude_from_testset) \
                                    or (dev and location_in_devset and not directory_info.exclude_from_testset) \
                                    or (not test and not dev and not location_in_testset and not location_in_devset):
                                self.example_indices.append((directory_info,int(idx_str)))

                            if not directory_info.silent:
                                location = (info['book'], info['sentence_index'])
                                self.voiced_data_locations[location] = (directory_info,int(idx_str))

        self.example_indices.sort()
        random.seed(0)
        random.shuffle(self.example_indices)

        self.no_normalizers = no_normalizers
        if not self.no_normalizers:
            self.mfcc_norm, self.emg_norm = pickle.load(open(FLAGS.normalizers_file,'rb'))

        sample_mfccs, sample_emg, _, _, _, _ = load_utterance(self.example_indices[0][0].directory, self.example_indices[0][1])
        self.num_speech_features = sample_mfccs.shape[1]
        self.num_features = sample_emg.shape[1]
        self.limit_length = limit_length
        self.num_sessions = len(directories)

        self.text_transform = TextTransform()

    def silent_subset(self):
        result = copy(self)
        silent_indices = []
        for example in self.example_indices:
            if example[0].silent:
                silent_indices.append(example)
        result.example_indices = silent_indices
        return result

    def subset(self, fraction):
        result = copy(self)
        result.example_indices = self.example_indices[:int(fraction*len(self.example_indices))]
        return result

    def __len__(self):
        return len(self.example_indices)

    @lru_cache(maxsize=None)
    def __getitem__(self, i):
        directory_info, idx = self.example_indices[i]
        mfccs, emg, text, book_location, phonemes, raw_emg = load_utterance(directory_info.directory, idx, self.limit_length, text_align_directory=self.text_align_directory)
        raw_emg = raw_emg / 20
        raw_emg = 50*np.tanh(raw_emg/50.)

        if not self.no_normalizers:
            mfccs = self.mfcc_norm.normalize(mfccs)
            emg = self.emg_norm.normalize(emg)
            emg = 8*np.tanh(emg/8.)

        session_ids = np.full(emg.shape[0], directory_info.session_index, dtype=np.int64)
        audio_file = f'{directory_info.directory}/{idx}_audio_clean.flac'

        text_int = np.array(self.text_transform.text_to_int(text), dtype=np.int64)

        result = {'audio_features':torch.from_numpy(mfccs).pin_memory(), 'emg':torch.from_numpy(emg).pin_memory(), 'text':text, 'text_int': torch.from_numpy(text_int).pin_memory(), 'file_label':idx, 'session_ids':torch.from_numpy(session_ids).pin_memory(), 'book_location':book_location, 'silent':directory_info.silent, 'raw_emg':torch.from_numpy(raw_emg).pin_memory()}

        if directory_info.silent:
            voiced_directory, voiced_idx = self.voiced_data_locations[book_location]
            voiced_mfccs, voiced_emg, _, _, phonemes, _ = load_utterance(voiced_directory.directory, voiced_idx, False, text_align_directory=self.text_align_directory)

            if not self.no_normalizers:
                voiced_mfccs = self.mfcc_norm.normalize(voiced_mfccs)
                voiced_emg = self.emg_norm.normalize(voiced_emg)
                voiced_emg = 8*np.tanh(voiced_emg/8.)

            result['parallel_voiced_audio_features'] = torch.from_numpy(voiced_mfccs).pin_memory()
            result['parallel_voiced_emg'] = torch.from_numpy(voiced_emg).pin_memory()

            audio_file = f'{voiced_directory.directory}/{voiced_idx}_audio_clean.flac'

        result['phonemes'] = torch.from_numpy(phonemes).pin_memory() # either from this example if vocalized or aligned example if silent
        result['audio_file'] = audio_file

        return result

    @staticmethod
    def collate_raw(batch):
        batch_size = len(batch)
        audio_features = []
        audio_feature_lengths = []
        parallel_emg = []
        for ex in batch:
            if ex['silent']:
                audio_features.append(ex['parallel_voiced_audio_features'])
                audio_feature_lengths.append(ex['parallel_voiced_audio_features'].shape[0])
                parallel_emg.append(ex['parallel_voiced_emg'])
            else:
                audio_features.append(ex['audio_features'])
                audio_feature_lengths.append(ex['audio_features'].shape[0])
                parallel_emg.append(np.zeros(1))
        phonemes = [ex['phonemes'] for ex in batch]
        emg = [ex['emg'] for ex in batch]
        raw_emg = [ex['raw_emg'] for ex in batch]
        session_ids = [ex['session_ids'] for ex in batch]
        lengths = [ex['emg'].shape[0] for ex in batch]
        silent = [ex['silent'] for ex in batch]
        text_ints = [ex['text_int'] for ex in batch]
        text_lengths = [ex['text_int'].shape[0] for ex in batch]

        result = {'audio_features':audio_features,
                  'audio_feature_lengths':audio_feature_lengths,
                  'emg':emg,
                  'raw_emg':raw_emg,
                  'parallel_voiced_emg':parallel_emg,
                  'phonemes':phonemes,
                  'session_ids':session_ids,
                  'lengths':lengths,
                  'silent':silent,
                  'text_int':text_ints,
                  'text_int_lengths':text_lengths}
        return result

def make_normalizers():
    dataset = EMGDataset(no_normalizers=True)
    mfcc_samples = []
    emg_samples = []
    for d in dataset:
        mfcc_samples.append(d['audio_features'])
        emg_samples.append(d['emg'])
        if len(emg_samples) > 50:
            break
    mfcc_norm = FeatureNormalizer(mfcc_samples, share_scale=True)
    emg_norm = FeatureNormalizer(emg_samples, share_scale=False)
    pickle.dump((mfcc_norm, emg_norm), open(FLAGS.normalizers_file, 'wb'))

if __name__ == '__main__':
    d = EMGDataset()
    for i in range(1000):
        d[i]


In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!cd ctcdecode && pip install .

In [ ]:
%%writefile convolution.py

import torch
import torch.nn as nn
from torch import Tensor
from typing import Tuple


class MaskCNN(nn.Module):
    r"""
    Masking Convolutional Neural Network

    Adds padding to the output of the module based on the given lengths.
    This is to ensure that the results of the model do not change when batch sizes change during inference.
    Input needs to be in the shape of (batch_size, channel, hidden_dim, seq_len)

    Refer to https://github.com/SeanNaren/deepspeech.pytorch/blob/master/model.py
    Copyright (c) 2017 Sean Naren
    MIT License

    Args:
        sequential (torch.nn): sequential list of convolution layer

    Inputs: inputs, seq_lengths
        - **inputs** (torch.FloatTensor): The input of size BxCxHxT
        - **seq_lengths** (torch.IntTensor): The actual length of each sequence in the batch

    Returns: output, seq_lengths
        - **output**: Masked output from the sequential
        - **seq_lengths**: Sequence length of output from the sequential
    """
    def __init__(self, sequential: nn.Sequential) -> None:
        super(MaskCNN, self).__init__()
        self.sequential = sequential

    def forward(self, inputs: Tensor, seq_lengths: Tensor) -> Tuple[Tensor, Tensor]:
        output = None

        for module in self.sequential:
            output = module(inputs)
            mask = torch.BoolTensor(output.size()).fill_(0)
            
            if output.is_cuda:
                mask = mask.cuda()
            
            print('mask size: ', mask.size())
            
            seq_lengths = self._get_sequence_lengths(module, seq_lengths)

            length = seq_lengths

            #if (mask[idx].size(2) - length) > 0:
                #mask[idx].narrow(dim=2, start=length, length=mask[idx].size(2) - length).fill_(1)

            output = output.masked_fill(mask, 0)
            inputs = output
            

        return output, seq_lengths

    def _get_sequence_lengths(self, module: nn.Module, seq_lengths: Tensor) -> Tensor:
        r"""
        Calculate convolutional neural network receptive formula

        Args:
            module (torch.nn.Module): module of CNN
            seq_lengths (torch.IntTensor): The actual length of each sequence in the batch

        Returns: seq_lengths
            - **seq_lengths**: Sequence length of output from the module
        """
        #print('seq_lengths 1: ', seq_lengths)
        if isinstance(module, nn.Conv2d):
            numerator = seq_lengths + 2 * module.padding[1] - module.dilation[1] * (module.kernel_size[1] - 1) - 1
            seq_lengths = numerator / float(module.stride[1])
            seq_lengths = round(seq_lengths) + 1
            #print('seq_lengths 2: ', seq_lengths)

        elif isinstance(module, nn.MaxPool2d):
            seq_lengths >>= 1
            #print('seq_lengths 3: ', seq_lengths)

        return seq_lengths


class VGGExtractor(nn.Module):
    r"""
    VGG extractor for automatic speech recognition described in
    "Advances in Joint CTC-Attention based End-to-End Speech Recognition with a Deep CNN Encoder and RNN-LM" paper
    - https://arxiv.org/pdf/1706.02737.pdf

    Args:
        input_dim (int): Dimension of input vector
        in_channels (int): Number of channels in the input image
        out_channels (int or tuple): Number of channels produced by the convolution

    Inputs: inputs, input_lengths
        - **inputs** (batch, time, dim): Tensor containing input vectors
        - **input_lengths**: Tensor containing containing sequence lengths

    Returns: outputs, output_lengths
        - **outputs**: Tensor produced by the convolution
        - **output_lengths**: Tensor containing sequence lengths produced by the convolution
    """
    def __init__(
            self,
            input_dim: int,
            in_channels: int = 1,
            out_channels: int or tuple = (64, 128),
    ):
        super(VGGExtractor, self).__init__()
        self.input_dim = input_dim
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.conv = MaskCNN(
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels[0], kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(num_features=out_channels[0]),
                nn.ReLU(),
                nn.Conv2d(out_channels[0], out_channels[0], kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(num_features=out_channels[0]),
                nn.ReLU(),
                nn.MaxPool2d(2, stride=2),
                nn.Conv2d(out_channels[0], out_channels[1], kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(num_features=out_channels[1]),
                nn.ReLU(),
                nn.Conv2d(out_channels[1], out_channels[1], kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(num_features=out_channels[1]),
                nn.ReLU(),
                nn.MaxPool2d(2, stride=2),
            )
        )

    def get_output_lengths(self, seq_lengths: Tensor):
        assert self.conv is not None, "self.conv should be defined"

        for module in self.conv:
            if isinstance(module, nn.Conv2d):
                numerator = seq_lengths + 2 * module.padding[1] - module.dilation[1] * (module.kernel_size[1] - 1) - 1
                seq_lengths = numerator.float() / float(module.stride[1])
                seq_lengths = seq_lengths.int() + 1

            elif isinstance(module, nn.MaxPool2d):
                seq_lengths >>= 1

        return seq_lengths.int()

    def get_output_dim(self):
        return round(self.input_dim*2.29)

    def forward(self, inputs: Tensor, input_lengths: Tensor) -> Tuple[Tensor, Tensor]:
        r"""
        inputs: torch.FloatTensor (batch, time, dimension)
        input_lengths: torch.IntTensor (batch)
        """
        outputs, output_lengths = self.conv(inputs.unsqueeze(1).transpose(2, 3), input_lengths)

        batch_size, channels, dimension, seq_lengths = outputs.size()
        outputs = outputs.permute(0, 3, 1, 2)
        outputs = outputs.view(batch_size, seq_lengths, channels * dimension)

        return outputs, output_lengths

In [ ]:
%%writefile embeddings.py

import math
import torch
import torch.nn as nn
from torch import Tensor


class PositionalEncoding(nn.Module):
    """
    Positional Encoding proposed in "Attention Is All You Need".
    Since speech_transformer contains no recurrence and no convolution, in order for the model to make
    use of the order of the sequence, we must add some positional information.

    "Attention Is All You Need" use sine and cosine functions of different frequencies:
        PE_(pos, 2i)    =  sin(pos / power(10000, 2i / d_model))
        PE_(pos, 2i+1)  =  cos(pos / power(10000, 2i / d_model))
    """
    def __init__(self, d_model: int = 512, max_len: int = 5000) -> None:
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model, requires_grad=False)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, length: int) -> Tensor:
        return self.pe[:, :length]


class Embedding(nn.Module):
    """
    Embedding layer. Similarly to other sequence transduction models, speech_transformer use learned embeddings
    to convert the input tokens and output tokens to vectors of dimension d_model.
    In the embedding layers, speech_transformer multiply those weights by sqrt(d_model)
    """
    def __init__(self, num_embeddings: int, pad_id: int, d_model: int = 512) -> None:
        super(Embedding, self).__init__()
        self.sqrt_dim = math.sqrt(d_model)
        self.embedding = nn.Embedding(num_embeddings, d_model, padding_idx=pad_id)

    def forward(self, inputs: Tensor) -> Tensor:
        return self.embedding(inputs) * self.sqrt_dim

In [ ]:
%%writefile mask.py

import torch

from torch import Tensor


def get_attn_pad_mask(inputs, input_lengths, expand_length):
    """ mask position is set to 1 """
    def get_transformer_non_pad_mask(inputs: Tensor, input_lengths: Tensor) -> Tensor:
        """ Padding position is set to 0, either use input_lengths or pad_id """
        batch_size = inputs.size(0)

        if isinstance(input_lengths, int):
            input_lengths = torch.tensor([input_lengths] * batch_size, dtype=torch.long, device=inputs.device)

        if len(inputs.size()) == 2:
            non_pad_mask = inputs.new_ones(inputs.size())  # B x T
        elif len(inputs.size()) == 3:
            non_pad_mask = inputs.new_ones(inputs.size()[:-1])  # B x T
        else:
            raise ValueError(f"Unsupported input shape {inputs.size()}")

        for i in range(batch_size):
            length = input_lengths[i].item()  # Extrair o valor inteiro
            non_pad_mask[i, length:] = 0

        return non_pad_mask


    non_pad_mask = get_transformer_non_pad_mask(inputs, input_lengths)
    pad_mask = non_pad_mask.lt(1)
    attn_pad_mask = pad_mask.unsqueeze(1).expand(-1, expand_length, -1)
    return attn_pad_mask


def get_attn_subsequent_mask(seq):
    assert seq.dim() == 2
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1)

    if seq.is_cuda:
        subsequent_mask = subsequent_mask.cuda()

    return subsequent_mask

In [ ]:
%%writefile modules.py

import torch
import torch.nn as nn
import torch.nn.init as init

from torch import Tensor
from typing import Tuple


class MaskConv2d(nn.Module):
    """
    Masking Convolutional Neural Network
    Adds padding to the output of the module based on the given lengths.
    This is to ensure that the results of the model do not change when batch sizes change during inference.
    Input needs to be in the shape of (batch_size, channel, hidden_dim, seq_len)
    Refer to https://github.com/SeanNaren/deepspeech.pytorch/blob/master/model.py
    Copyright (c) 2017 Sean Naren
    MIT License
    Args:
        sequential (torch.nn): sequential list of convolution layer
    Inputs: inputs, seq_lengths
        - **inputs** (torch.FloatTensor): The input of size BxCxHxT
        - **seq_lengths** (torch.IntTensor): The actual length of each sequence in the batch
    Returns: output, seq_lengths
        - **output**: Masked output from the sequential
        - **seq_lengths**: Sequence length of output from the sequential
    """
    def __init__(self, sequential: nn.Sequential) -> None:
        super(MaskConv2d, self).__init__()
        self.sequential = sequential

    def forward(self, inputs: Tensor, seq_lengths: Tensor) -> Tuple[Tensor, Tensor]:
        output = None

        for module in self.sequential:
            output = module(inputs)
            mask = torch.BoolTensor(output.size()).fill_(0)

            if output.is_cuda:
                mask = mask.cuda()

            seq_lengths = self.get_sequence_lengths(module, seq_lengths)
            length = seq_lengths

            if (mask[idx].size(2) - length) > 0:
                mask[idx].narrow(dim=2, start=length, length=mask[idx].size(2) - length).fill_(1)

            output = output.masked_fill(mask, 0)
            inputs = output

        return output, seq_lengths

    def get_sequence_lengths(self, module: nn.Module, seq_lengths: Tensor) -> Tensor:
        """
        Calculate convolutional neural network receptive formula
        Args:
            module (torch.nn.Module): module of CNN
            seq_lengths (torch.IntTensor): The actual length of each sequence in the batch
        Returns: seq_lengths
            - **seq_lengths**: Sequence length of output from the module
        """
        if isinstance(module, nn.Conv2d):
            numerator = seq_lengths + 2 * module.padding[1] - module.dilation[1] * (module.kernel_size[1] - 1) - 1
            seq_lengths = numerator.float() / float(module.stride[1])
            seq_lengths = seq_lengths.int() + 1

        elif isinstance(module, nn.MaxPool2d):
            seq_lengths >>= 1

        return seq_lengths.int()


class Linear(nn.Module):
    """
    Wrapper class of torch.nn.Linear
    Weight initialize by xavier initialization and bias initialize to zeros.
    """
    def __init__(self, in_features: int, out_features: int, bias: bool = True) -> None:
        super(Linear, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        init.xavier_uniform_(self.linear.weight)
        if bias:
            init.zeros_(self.linear.bias)

    def forward(self, x: Tensor) -> Tensor:
        #print('x: ', x)
        #print('x size: ', x.size())
        return self.linear(x)


class Transpose(nn.Module):
    """ Wrapper class of torch.transpose() for Sequential module. """
    def __init__(self, shape: tuple):
        super(Transpose, self).__init__()
        self.shape = shape

    def forward(self, inputs: Tensor):
        return inputs.transpose(*self.shape)

In [ ]:
%%writefile sublayers.py

import torch.nn as nn

from torch import Tensor
from typing import Any, Optional
from modules import (
    Linear,
    MaskConv2d,
)


class AddNorm(nn.Module):
    """
    Add & Normalization layer proposed in "Attention Is All You Need".
    Transformer employ a residual connection around each of the two sub-layers,
    (Multi-Head Attention & Feed-Forward) followed by layer normalization.
    """
    def __init__(self, sublayer: nn.Module, d_model: int = 512) -> None:
        super(AddNorm, self).__init__()
        self.sublayer = sublayer
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, *args):
        residual = args[0]
        output = self.sublayer(*args)

        if isinstance(output, tuple):
            return self.layer_norm(output[0] + residual), output[1]

        return self.layer_norm(output + residual)


class PositionWiseFeedForwardNet(nn.Module):
    """
    Position-wise Feedforward Networks proposed in "Attention Is All You Need".
    Fully connected feed-forward network, which is applied to each position separately and identically.
    This consists of two linear transformations with a ReLU activation in between.
    Another way of describing this is as two convolutions with kernel size 1.
    """
    def __init__(self, d_model: int = 512, d_ff: int = 2048,
                 dropout_p: float = 0.3, ffnet_style: str = 'ff') -> None:
        super(PositionWiseFeedForwardNet, self).__init__()
        self.ffnet_style = ffnet_style.lower()
        if self.ffnet_style == 'ff':
            self.feed_forward = nn.Sequential(
                Linear(d_model, d_ff),
                nn.Dropout(dropout_p),
                nn.ReLU(),
                Linear(d_ff, d_model),
                nn.Dropout(dropout_p),
            )

        elif self.ffnet_style == 'conv':
            self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
            self.relu = nn.ReLU()
            self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)

        else:
            raise ValueError("Unsupported mode: {0}".format(self.mode))

    def forward(self, inputs: Tensor) -> Tensor:
        if self.ffnet_style == 'conv':
            output = self.conv1(inputs.transpose(1, 2))
            output = self.relu(output)
            return self.conv2(output).transpose(1, 2)

        return self.feed_forward(inputs)

In [ ]:
%%writefile attention.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from modules import Linear
from torch import Tensor
from typing import Optional, Tuple


class ScaledDotProductAttention(nn.Module):
    """
    Scaled Dot-Product Attention proposed in "Attention Is All You Need"
    Compute the dot products of the query with all keys, divide each by sqrt(dim),
    and apply a softmax function to obtain the weights on the values

    Args: dim, mask
        dim (int): dimension of attention
        mask (torch.Tensor): tensor containing indices to be masked

    Inputs: query, key, value, mask
        - **query** (batch, q_len, d_model): tensor containing projection vector for decoder.
        - **key** (batch, k_len, d_model): tensor containing projection vector for encoder.
        - **value** (batch, v_len, d_model): tensor containing features of the encoded input sequence.
        - **mask** (-): tensor containing indices to be masked

    Returns: context, attn
        - **context**: tensor containing the context vector from attention mechanism.
        - **attn**: tensor containing the attention (alignment) from the encoder outputs.
    """
    def __init__(self, dim: int) -> None:
        super(ScaledDotProductAttention, self).__init__()
        self.sqrt_dim = np.sqrt(dim)

    def forward(self, query: Tensor, key: Tensor, value: Tensor, mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        score = torch.bmm(query, key.transpose(1, 2)) / self.sqrt_dim

        if mask is not None:
            score.masked_fill_(mask, -1e9)

        attn = F.softmax(score, -1)
        context = torch.bmm(attn, value)
        return context, attn


class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention proposed in "Attention Is All You Need"
    Instead of performing a single attention function with d_model-dimensional keys, values, and queries,
    project the queries, keys and values h times with different, learned linear projections to d_head dimensions.
    These are concatenated and once again projected, resulting in the final values.
    Multi-head attention allows the model to jointly attend to information from different representation
    subspaces at different positions.

    MultiHead(Q, K, V) = Concat(head_1, ..., head_h) · W_o
        where head_i = Attention(Q · W_q, K · W_k, V · W_v)

    Args:
        d_model (int): The dimension of keys / values / quries (default: 512)
        num_heads (int): The number of attention heads. (default: 8)

    Inputs: query, key, value, mask
        - **query** (batch, q_len, d_model): tensor containing projection vector for decoder.
        - **key** (batch, k_len, d_model): tensor containing projection vector for encoder.
        - **value** (batch, v_len, d_model): tensor containing features of the encoded input sequence.
        - **mask** (-): tensor containing indices to be masked

    Returns: output, attn
        - **output** (batch, output_len, dimensions): tensor containing the attended output features.
        - **attn** (batch * num_heads, v_len): tensor containing the attention (alignment) from the encoder outputs.
    """
    def __init__(self, d_model: int = 512, num_heads: int = 8) -> None:
        super(MultiHeadAttention, self).__init__()

        assert d_model % num_heads == 0, "hidden_dim % num_heads should be zero."

        self.d_head = int(d_model / num_heads)
        self.num_heads = num_heads
        self.query_proj = Linear(d_model, self.d_head * num_heads)
        self.key_proj = Linear(d_model, self.d_head * num_heads)
        self.value_proj = Linear(d_model, self.d_head * num_heads)
        self.sqrt_dim = np.sqrt(d_model)
        self.scaled_dot_attn = ScaledDotProductAttention(self.d_head)

    def forward(self, query: Tensor, key: Tensor, value: Tensor, mask: Optional[Tensor] = None) -> Tuple[Tensor, Tensor]:
        batch_size = value.size(0)

        query = self.query_proj(query).view(batch_size, -1, self.num_heads, self.d_head)  # BxQ_LENxNxD
        key = self.key_proj(key).view(batch_size, -1, self.num_heads, self.d_head)        # BxK_LENxNxD
        value = self.value_proj(value).view(batch_size, -1, self.num_heads, self.d_head)  # BxV_LENxNxD

        query = query.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxQ_LENxD
        key = key.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)      # BNxK_LENxD
        value = value.permute(2, 0, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.d_head)  # BNxV_LENxD

        if mask is not None:
            mask = mask.repeat(self.num_heads, 1, 1)

        context, attn = self.scaled_dot_attn(query, key, value, mask)
        context = context.view(self.num_heads, batch_size, -1, self.d_head)
        context = context.permute(1, 2, 0, 3).contiguous().view(batch_size, -1, self.num_heads * self.d_head)  # BxTxND

        return context, attn

In [ ]:
%%writefile decoder.py

import torch
import torch.nn as nn
import random
from torch import Tensor
from typing import Optional, Any, Tuple

from attention import MultiHeadAttention
from embeddings import Embedding, PositionalEncoding
from mask import get_attn_pad_mask, get_attn_subsequent_mask
from modules import Linear
from sublayers import AddNorm, PositionWiseFeedForwardNet


class SpeechTransformerDecoderLayer(nn.Module):
    """
    DecoderLayer is made up of self-attention, multi-head attention and feedforward network.
    This standard decoder layer is based on the paper "Attention Is All You Need".

    Args:
        d_model: dimension of model (default: 512)
        num_heads: number of attention heads (default: 8)
        d_ff: dimension of feed forward network (default: 2048)
        dropout_p: probability of dropout (default: 0.3)
        ffnet_style: style of feed forward network [ff, conv] (default: ff)
    """

    def __init__(
            self,
            d_model: int = 512,             # dimension of model
            num_heads: int = 8,             # number of attention heads
            d_ff: int = 2048,               # dimension of feed forward network
            dropout_p: float = 0.3,         # probability of dropout
            ffnet_style: str = 'ff'         # style of feed forward network
    ) -> None:
        super(SpeechTransformerDecoderLayer, self).__init__()
        self.self_attention = AddNorm(MultiHeadAttention(d_model, num_heads), d_model)
        self.memory_attention = AddNorm(MultiHeadAttention(d_model, num_heads), d_model)
        self.feed_forward = AddNorm(PositionWiseFeedForwardNet(d_model, d_ff, dropout_p, ffnet_style), d_model)

    def forward(
            self,
            inputs: Tensor,
            encoder_outputs: Tensor,
            self_attn_mask: Optional[Any] = None,
            memory_mask: Optional[Any] = None
    ) -> Tuple[Tensor, Tensor, Tensor]:
        memory = encoder_outputs
        output, self_attn = self.self_attention(inputs, inputs, inputs, self_attn_mask)
        output, memory_attn = self.memory_attention(output, memory, memory, memory_mask)
        output = self.feed_forward(output)
        return output, self_attn, memory_attn


class SpeechTransformerDecoder(nn.Module):
    r"""
    The TransformerDecoder is composed of a stack of N identical layers.
    Each layer has three sub-layers. The first is a multi-head self-attention mechanism,
    and the second is a multi-head attention mechanism, third is a feed-forward network.

    Args:
        num_classes: umber of classes
        d_model: dimension of model
        d_ff: dimension of feed forward network
        num_layers: number of decoder layers
        num_heads: number of attention heads
        ffnet_style: style of feed forward network
        dropout_p: probability of dropout
        pad_id: identification of pad token
        eos_id: identification of end of sentence token
    """

    def __init__(
            self,
            num_classes: int,
            d_model: int = 512,
            d_ff: int = 2048,
            num_layers: int = 6,
            num_heads: int = 8,
            ffnet_style: str = 'ff',
            dropout_p: float = 0.3,
            pad_id: int = 0,
            sos_id: int = 1,
            eos_id: int = 2,
            max_length: int = 128,
    ) -> None:
        super(SpeechTransformerDecoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.embedding = Embedding(num_classes, pad_id, d_model)
        self.positional_encoding = PositionalEncoding(d_model)
        self.input_dropout = nn.Dropout(p=dropout_p)
        self.layers = nn.ModuleList([
            SpeechTransformerDecoderLayer(d_model, num_heads, d_ff, dropout_p, ffnet_style) for _ in range(num_layers)
        ])
        self.pad_id = pad_id
        self.sos_id = sos_id
        self.eos_id = eos_id
        self.max_length = max_length
        self.fc = nn.Sequential(
            nn.LayerNorm(d_model),
            Linear(d_model, num_classes, bias=False),
        )

    def forward_step(
            self,
            decoder_inputs,
            decoder_input_lengths,
            encoder_outputs,
            encoder_output_lengths,
            positional_encoding_length,
    ) -> Tensor:
        dec_self_attn_pad_mask = get_attn_pad_mask(
            decoder_inputs, decoder_input_lengths, decoder_inputs.size(1)
        )
        dec_self_attn_subsequent_mask = get_attn_subsequent_mask(decoder_inputs)
        self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequent_mask), 0)

        encoder_attn_mask = get_attn_pad_mask(
            encoder_outputs, encoder_output_lengths, decoder_inputs.size(1)
        )

        outputs = self.embedding(decoder_inputs) + self.positional_encoding(positional_encoding_length)
        outputs = self.input_dropout(outputs)

        for layer in self.layers:
            outputs, self_attn, memory_attn = layer(
                inputs=outputs,
                encoder_outputs=encoder_outputs,
                self_attn_mask=self_attn_mask,
                memory_mask=encoder_attn_mask,
            )

        return outputs

    def forward(
            self,
            encoder_outputs: Tensor,
            targets: Optional[torch.LongTensor] = None,
            encoder_output_lengths: Tensor = None,
            target_lengths: Tensor = None,
            teacher_forcing_ratio: float = 1.0,
    ) -> Tensor:
        r"""
        Forward propagate a `encoder_outputs` for training.

        Args:
            targets (torch.LongTensor): A target sequence passed to decoders. `IntTensor` of size
                ``(batch, seq_length)``
            encoder_outputs (torch.FloatTensor): A output sequence of encoders. `FloatTensor` of size
                ``(batch, seq_length, dimension)``
            encoder_output_lengths (torch.LongTensor): The length of encoders outputs. ``(batch)``
            teacher_forcing_ratio (float): ratio of teacher forcing

        Returns:
            * logits (torch.FloatTensor): Log probability of model predictions.
        """
        batch_size = encoder_outputs.size(0)
        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

        if targets is not None and use_teacher_forcing:
            targets = targets[targets != self.eos_id].view(batch_size, -1)
            target_length = targets.size(1)

            outputs = self.forward_step(
                decoder_inputs=targets,
                decoder_input_lengths=target_lengths,
                encoder_outputs=encoder_outputs,
                encoder_output_lengths=encoder_output_lengths,
                positional_encoding_length=target_length,
            )
            return self.fc(outputs).log_softmax(dim=-1)

        # Inference
        else:
            logits = list()

            input_var = encoder_outputs.new_zeros(batch_size, self.max_length).long()
            input_var = input_var.fill_(self.pad_id)
            input_var[:, 0] = self.sos_id

            for di in range(1, self.max_length):
                input_lengths = torch.IntTensor(batch_size).fill_(di)

                outputs = self.forward_step(
                    decoder_inputs=input_var[:, :di],
                    decoder_input_lengths=input_lengths,
                    encoder_outputs=encoder_outputs,
                    encoder_output_lengths=encoder_output_lengths,
                    positional_encoding_length=di,
                )
                step_output = self.fc(outputs).log_softmax(dim=-1)

                logits.append(step_output[:, -1, :])
                input_var = logits[-1].topk(1)[1]

            return torch.stack(logits, dim=1)

In [ ]:
%%writefile encoder.py

import torch.nn as nn
from torch import Tensor
from typing import Tuple, Optional, Any

from attention import MultiHeadAttention
from convolution import VGGExtractor
from embeddings import PositionalEncoding
from mask import get_attn_pad_mask
from modules import Linear
from sublayers import AddNorm, PositionWiseFeedForwardNet


class SpeechTransformerEncoderLayer(nn.Module):
    """
    EncoderLayer is made up of self-attention and feedforward network.
    This standard encoder layer is based on the paper "Attention Is All You Need".

    Args:
        d_model: dimension of model (default: 512)
        num_heads: number of attention heads (default: 8)
        d_ff: dimension of feed forward network (default: 2048)
        dropout_p: probability of dropout (default: 0.3)
        ffnet_style: style of feed forward network [ff, conv] (default: ff)
    """

    def __init__(
            self,
            d_model: int = 512,             # dimension of model
            num_heads: int = 8,             # number of attention heads
            d_ff: int = 2048,               # dimension of feed forward network
            dropout_p: float = 0.3,         # probability of dropout
            ffnet_style: str = 'ff'         # style of feed forward network
    ) -> None:
        super(SpeechTransformerEncoderLayer, self).__init__()
        self.self_attention = AddNorm(MultiHeadAttention(d_model, num_heads), d_model)
        self.feed_forward = AddNorm(PositionWiseFeedForwardNet(d_model, d_ff, dropout_p, ffnet_style), d_model)

    def forward(self, inputs: Tensor, self_attn_mask: Optional[Any] = None) -> Tuple[Tensor, Tensor]:
        output, attn = self.self_attention(inputs, inputs, inputs, self_attn_mask)
        output = self.feed_forward(output)
        return output, attn


class SpeechTransformerEncoder(nn.Module):
    """
    The TransformerEncoder is composed of a stack of N identical layers.
    Each layer has two sub-layers. The first is a multi-head self-attention mechanism,
    and the second is a simple, position-wise fully connected feed-forward network.

    Args:
        d_model: dimension of model (default: 512)
        input_dim: dimension of feature vector (default: 80)
        d_ff: dimension of feed forward network (default: 2048)
        num_layers: number of encoder layers (default: 6)
        num_heads: number of attention heads (default: 8)
        ffnet_style: style of feed forward network [ff, conv] (default: ff)
        dropout_p:  probability of dropout (default: 0.3)
        pad_id: identification of pad token (default: 0)

    Inputs:
        - **inputs**: list of sequences, whose length is the batch size and within which each sequence is list of tokens
        - **input_lengths**: list of sequence lengths
    """

    def __init__(
            self,
            d_model: int = 512,
            input_dim: int = 80,
            d_ff: int = 2048,
            num_layers: int = 6,
            num_heads: int = 8,
            ffnet_style: str = 'ff',
            dropout_p: float = 0.3,
            pad_id: int = 0,
    ) -> None:
        super(SpeechTransformerEncoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.pad_id = pad_id
        self.conv = VGGExtractor(input_dim)
        self.input_proj = Linear(self.conv.get_output_dim(), d_model)
        self.input_dropout = nn.Dropout(p=dropout_p)
        self.positional_encoding = PositionalEncoding(d_model)
        self.layers = nn.ModuleList(
            [SpeechTransformerEncoderLayer(d_model, num_heads, d_ff, dropout_p, ffnet_style) for _ in range(num_layers)]
        )

    def forward(self, inputs: Tensor, input_lengths: Tensor = None) -> Tuple[Tensor, Tensor]:
        conv_outputs, output_lengths = self.conv(inputs, input_lengths)

        self_attn_mask = get_attn_pad_mask(conv_outputs, output_lengths, conv_outputs.size(1))
        print('conv_outputs: ', conv_outputs.size())
        outputs = self.input_proj(conv_outputs)
        outputs += self.positional_encoding(outputs.size(1))
        outputs = self.input_dropout(outputs)

        for layer in self.layers:
            outputs, attn = layer(outputs, self_attn_mask)

        return outputs, output_lengths

In [ ]:
%%writefile beam_decoder.py

import torch
import torch.nn as nn
from torch import Tensor

from decoder import SpeechTransformerDecoder


class BeamTransformerDecoder(nn.Module):
    def __init__(self, decoder: SpeechTransformerDecoder, batch_size: int, beam_size: int = 3) -> None:
        super(BeamTransformerDecoder, self).__init__()
        self.decoder = decoder
        self.beam_size = beam_size
        self.sos_id = decoder.sos_id
        self.pad_id = decoder.pad_id
        self.eos_id = decoder.eos_id
        self.ongoing_beams = None
        self.cumulative_ps = None
        self.finished = [[] for _ in range(batch_size)]
        self.finished_ps = [[] for _ in range(batch_size)]
        self.forward_step = decoder.forward_step
        self.use_cuda = True if torch.cuda.is_available() else False

    def _inflate(self, tensor: Tensor, n_repeat: int, dim: int) -> Tensor:
        repeat_dims = [1] * len(tensor.size())
        repeat_dims[dim] *= n_repeat

        return tensor.repeat(*repeat_dims)

    def _get_successor(
            self,
            current_ps: Tensor,
            current_vs: Tensor,
            finished_ids: tuple,
            num_successor: int,
            eos_count: int,
            k: int
    ) -> int:
        finished_batch_idx, finished_idx = finished_ids

        successor_ids = current_ps.topk(k + num_successor)[1]
        successor_idx = successor_ids[finished_batch_idx, -1]

        successor_p = current_ps[finished_batch_idx, successor_idx]
        successor_v = current_vs[finished_batch_idx, successor_idx]

        prev_status_idx = (successor_idx // k)
        prev_status = self.ongoing_beams[finished_batch_idx, prev_status_idx]
        prev_status = prev_status.view(-1)[:-1]

        successor = torch.cat([prev_status, successor_v.view(1)])

        if int(successor_v) == self.eos_id:
            self.finished[finished_batch_idx].append(successor)
            self.finished_ps[finished_batch_idx].append(successor_p)
            eos_count = self._get_successor(
                current_ps=current_ps,
                current_vs=current_vs,
                finished_ids=finished_ids,
                num_successor=num_successor + eos_count,
                eos_count=eos_count + 1,
                k=k,
            )

        else:
            self.ongoing_beams[finished_batch_idx, finished_idx] = successor
            self.cumulative_ps[finished_batch_idx, finished_idx] = successor_p

        return eos_count

    def _get_hypothesis(self):
        predictions = list()

        for batch_idx, batch in enumerate(self.finished):
            # if there is no terminated sentences, bring ongoing sentence which has the highest probability instead
            if len(batch) == 0:
                prob_batch = self.cumulative_ps[batch_idx]
                top_beam_idx = int(prob_batch.topk(1)[1])
                predictions.append(self.ongoing_beams[batch_idx, top_beam_idx])

            # bring highest probability sentence
            else:
                top_beam_idx = int(torch.FloatTensor(self.finished_ps[batch_idx]).topk(1)[1])
                predictions.append(self.finished[batch_idx][top_beam_idx])

        predictions = self._fill_sequence(predictions)
        return predictions

    def _is_all_finished(self, k: int) -> bool:
        for done in self.finished:
            if len(done) < k:
                return False

        return True

    def _fill_sequence(self, y_hats: list) -> Tensor:
        batch_size = len(y_hats)
        max_length = -1

        for y_hat in y_hats:
            if len(y_hat) > max_length:
                max_length = len(y_hat)

        matched = torch.zeros((batch_size, max_length), dtype=torch.long)

        for batch_idx, y_hat in enumerate(y_hats):
            matched[batch_idx, :len(y_hat)] = y_hat
            matched[batch_idx, len(y_hat):] = int(self.pad_id)

        return matched

    def forward(self, encoder_outputs: torch.FloatTensor, encoder_output_lengths: torch.FloatTensor):
        batch_size = encoder_outputs.size(0)

        decoder_inputs = torch.IntTensor(batch_size, self.decoder.max_length).fill_(self.sos_id).long()
        decoder_input_lengths = torch.IntTensor(batch_size).fill_(1)

        outputs = self.forward_step(
            decoder_inputs=decoder_inputs[:, :1],
            decoder_input_lengths=decoder_input_lengths,
            encoder_outputs=encoder_outputs,
            encoder_output_lengths=encoder_output_lengths,
            positional_encoding_length=1,
        )
        step_outputs = self.decoder.fc(outputs).log_softmax(dim=-1)
        self.cumulative_ps, self.ongoing_beams = step_outputs.topk(self.beam_size)

        self.ongoing_beams = self.ongoing_beams.view(batch_size * self.beam_size, 1)
        self.cumulative_ps = self.cumulative_ps.view(batch_size * self.beam_size, 1)

        decoder_inputs = torch.IntTensor(batch_size * self.beam_size, 1).fill_(self.sos_id)
        decoder_inputs = torch.cat((decoder_inputs, self.ongoing_beams), dim=-1)  # bsz * beam x 2

        encoder_dim = encoder_outputs.size(2)
        encoder_outputs = self._inflate(encoder_outputs, self.beam_size, dim=0)
        encoder_outputs = encoder_outputs.view(self.beam_size, batch_size, -1, encoder_dim)
        encoder_outputs = encoder_outputs.transpose(0, 1)
        encoder_outputs = encoder_outputs.reshape(batch_size * self.beam_size, -1, encoder_dim)

        encoder_output_lengths = encoder_output_lengths.unsqueeze(1).repeat(1, self.beam_size).view(-1)

        for di in range(2, self.decoder.max_length):
            if self._is_all_finished(self.beam_size):
                break

            decoder_input_lengths = torch.LongTensor(batch_size * self.beam_size).fill_(di)

            step_outputs = self.forward_step(
                decoder_inputs=decoder_inputs[:, :di],
                decoder_input_lengths=decoder_input_lengths,
                encoder_outputs=encoder_outputs,
                encoder_output_lengths=encoder_output_lengths,
                positional_encoding_length=di,
            )
            step_outputs = self.decoder.fc(step_outputs).log_softmax(dim=-1)

            step_outputs = step_outputs.view(batch_size, self.beam_size, -1, 10)
            current_ps, current_vs = step_outputs.topk(self.beam_size)

            # TODO: Check transformer's beam search
            current_ps = current_ps[:, :, -1, :]
            current_vs = current_vs[:, :, -1, :]

            self.cumulative_ps = self.cumulative_ps.view(batch_size, self.beam_size)
            self.ongoing_beams = self.ongoing_beams.view(batch_size, self.beam_size, -1)

            current_ps = (current_ps.permute(0, 2, 1) + self.cumulative_ps.unsqueeze(1)).permute(0, 2, 1)
            current_ps = current_ps.view(batch_size, self.beam_size ** 2)
            current_vs = current_vs.contiguous().view(batch_size, self.beam_size ** 2)

            self.cumulative_ps = self.cumulative_ps.view(batch_size, self.beam_size)
            self.ongoing_beams = self.ongoing_beams.view(batch_size, self.beam_size, -1)

            topk_current_ps, topk_status_ids = current_ps.topk(self.beam_size)
            prev_status_ids = (topk_status_ids // self.beam_size)

            topk_current_vs = torch.zeros((batch_size, self.beam_size), dtype=torch.long)
            prev_status = torch.zeros(self.ongoing_beams.size(), dtype=torch.long)

            for batch_idx, batch in enumerate(topk_status_ids):
                for idx, topk_status_idx in enumerate(batch):
                    topk_current_vs[batch_idx, idx] = current_vs[batch_idx, topk_status_idx]
                    prev_status[batch_idx, idx] = self.ongoing_beams[batch_idx, prev_status_ids[batch_idx, idx]]

            self.ongoing_beams = torch.cat([prev_status, topk_current_vs.unsqueeze(2)], dim=2)
            self.cumulative_ps = topk_current_ps

            if torch.any(topk_current_vs == self.eos_id):
                finished_ids = torch.where(topk_current_vs == self.eos_id)
                num_successors = [1] * batch_size

                for (batch_idx, idx) in zip(*finished_ids):
                    self.finished[batch_idx].append(self.ongoing_beams[batch_idx, idx])
                    self.finished_ps[batch_idx].append(self.cumulative_ps[batch_idx, idx])

                    if self.beam_size != 1:
                        eos_count = self._get_successor(
                            current_ps=current_ps,
                            current_vs=current_vs,
                            finished_ids=(batch_idx, idx),
                            num_successor=num_successors[batch_idx],
                            eos_count=1,
                            k=self.beam_size,
                        )
                        num_successors[batch_idx] += eos_count

            ongoing_beams = self.ongoing_beams.clone().view(batch_size * self.beam_size, -1)
            decoder_inputs = torch.cat((decoder_inputs, ongoing_beams[:, :-1]), dim=-1)

        return self._get_hypothesis()

In [ ]:
%%writefile model.py

import torch.nn as nn
from torch import Tensor
from typing import Optional, Union

from beam_decoder import BeamTransformerDecoder
from decoder import SpeechTransformerDecoder
from encoder import SpeechTransformerEncoder


class SpeechTransformer(nn.Module):
    """
    A Speech Transformer model. User is able to modify the attributes as needed.
    The model is based on the paper "Attention Is All You Need".

    Args:
        num_classes (int): the number of classfication
        d_model (int): dimension of model (default: 512)
        input_dim (int): dimension of input
        pad_id (int): identification of <PAD_token>
        eos_id (int): identification of <EOS_token>
        d_ff (int): dimension of feed forward network (default: 2048)
        num_encoder_layers (int): number of encoder layers (default: 6)
        num_decoder_layers (int): number of decoder layers (default: 6)
        num_heads (int): number of attention heads (default: 8)
        dropout_p (float): dropout probability (default: 0.3)
        ffnet_style (str): if poswise_ffnet is 'ff', position-wise feed forware network to be a feed forward,
            otherwise, position-wise feed forward network to be a convolution layer. (default: ff)

    Inputs: inputs, input_lengths, targets, teacher_forcing_ratio
        - **inputs** (torch.Tensor): tensor of sequences, whose length is the batch size and within which
          each sequence is a list of token IDs. This information is forwarded to the encoder.
        - **input_lengths** (torch.Tensor): tensor of sequences, whose contains length of inputs.
        - **targets** (torch.Tensor): tensor of sequences, whose length is the batch size and within which
          each sequence is a list of token IDs. This information is forwarded to the decoder.

    Returns: output
        - **output**: tensor containing the outputs
    """

    def __init__(
            self,
            num_classes: int,
            d_model: int = 512,
            input_dim: int = 80,
            pad_id: int = 0,
            sos_id: int = 1,
            eos_id: int = 2,
            d_ff: int = 2048,
            num_heads: int = 8,
            num_encoder_layers: int = 6,
            num_decoder_layers: int = 6,
            dropout_p: float = 0.3,
            ffnet_style: str = 'ff',
            extractor: str = 'vgg',
            joint_ctc_attention: bool = False,
            max_length: int = 128,
    ) -> None:
        super(SpeechTransformer, self).__init__()

        assert d_model % num_heads == 0, "d_model % num_heads should be zero."

        self.num_classes = num_classes
        self.extractor = extractor
        self.joint_ctc_attention = joint_ctc_attention
        self.sos_id = sos_id
        self.eos_id = eos_id
        self.pad_id = pad_id
        self.max_length = max_length

        self.encoder = SpeechTransformerEncoder(
            d_model=d_model,
            input_dim=input_dim,
            d_ff=d_ff,
            num_layers=num_encoder_layers,
            num_heads=num_heads,
            ffnet_style=ffnet_style,
            dropout_p=dropout_p,
            pad_id=pad_id,
        )

        self.decoder = SpeechTransformerDecoder(
            num_classes=num_classes,
            d_model=d_model,
            d_ff=d_ff,
            num_layers=num_decoder_layers,
            num_heads=num_heads,
            ffnet_style=ffnet_style,
            dropout_p=dropout_p,
            pad_id=pad_id,
            sos_id=sos_id,
            eos_id=eos_id,
            max_length = max_length,
        )

    def set_beam_decoder(self, batch_size: int = None, beam_size: int = 3):
        """ Setting beam search decoder """
        self.decoder = BeamTransformerDecoder(
            decoder=self.decoder,
            batch_size=batch_size,
            beam_size=beam_size,
        )

    def forward(
            self,
            inputs: Tensor,
            input_lengths: Tensor,
            targets: Optional[Tensor] = None,
            target_lengths: Optional[Tensor] = None,
    ) -> Union[Tensor, tuple]:
        """
        inputs (torch.FloatTensor): (batch_size, sequence_length, dimension)
        input_lengths (torch.LongTensor): (batch_size)
        """
        logits = None
        #print('inputs: ', inputs)
        #print('inputs lengths model: ', inputs.size())
        encoder_outputs, encoder_output_lengths = self.encoder(inputs, input_lengths)
        if isinstance(self.decoder, BeamTransformerDecoder):
            predictions = self.decoder(encoder_outputs, encoder_output_lengths)
        else:
            logits = self.decoder(
                encoder_outputs=encoder_outputs,
                encoder_output_lengths=encoder_output_lengths,
                targets=targets,
                teacher_forcing_ratio=0.0,
                target_lengths=target_lengths,
            )
            predictions = logits.max(-1)[1]

        return predictions, logits

In [ ]:
import os
import sys
import numpy as np
import logging
import subprocess
from ctcdecode import CTCBeamDecoder
import jiwer
import random
import gc

import torch
from torch import nn
import torch.nn.functional as F

from read_emg import EMGDataset, SizeAwareSampler
from data_utils import combine_fixed_length, decollate_tensor
from model import SpeechTransformer

#from torch.utils.checkpoint import checkpoint

from absl import flags
FLAGS = flags.FLAGS

gc.get_threshold()
gc.get_count()
gc.collect()
gc.get_count()

torch.cuda.empty_cache()
torch.cuda.set_per_process_memory_fraction(0.6, device=0)
torch.cuda.max_split_size_mb(512, device=0)


def test(model, testset, device):  #avalia o desempenho de um modelo em um conjunto de teste
    model.eval() #Define o modelo no modo de avaliação, desativando gradientes e camadas como o Dropout.

    blank_id = len(testset.text_transform.chars)
    decoder = CTCBeamDecoder(testset.text_transform.chars+'_', blank_id=blank_id, log_probs_input=True,  #CTCBeamDecoder é usado para realizar a decodificação do resultado do modelo em sequências de texto.
            model_path='/kaggle/input/librispeech-4gram-language-model/4-gram-librispeech.bin', alpha=1.5, beta=1.85)

    dataloader = torch.utils.data.DataLoader(testset, batch_size=1)
    references = []
    predictions = []
    with torch.no_grad():
        for example in dataloader:
            X = example['emg'].to(device)
            X_raw = example['raw_emg'].to(device)
            sess = example['session_ids'].to(device)
            
            xraw_lenght= X_raw.size(1)
            pred  = F.log_softmax(model(X_raw, xraw_lenght), -1)  #Aplica a função de softmax aos logits de saída do modelo e, em seguida, calcula o logaritmo das probabilidades resultantes.

            beam_results, beam_scores, timesteps, out_lens = decoder.decode(pred) #Usa o decodificador CTC para obter as sequências mais prováveis ​​a partir dos logits previstos pelo modelo.
            pred_int = beam_results[0,0,:out_lens[0,0]].tolist()

            pred_text = testset.text_transform.int_to_text(pred_int)  #Converte as previsões em formato de lista de inteiros em texto usando a função int_to_text da transformação de texto do conjunto de teste.
            #print('pred_text: ', pred_text)
            target_text = testset.text_transform.clean_text(example['text'][0])  #Obtém o texto verdadeiro correspondente ao exemplo atual do conjunto de teste.
            #print('target text: ',target_text)
            
            if len(target_text) != 0:
                references.append(target_text)
                #print('references: ', references)
                predictions.append(pred_text)
                #print('predictions: ', predictions)
                #print(len(references))
            else:
                continue
            # ^Adiciona as referências e as previsões às listas correspondentes.
    print('references: ', references)
    print('predictions: ', predictions)
    model.train() #Define o modelo de volta no modo de treinamento.
    return jiwer.wer(references, predictions) #Calcula a taxa de erro de palavra (WER) usando a biblioteca jiwer, comparando as referências e as previsões.



def train_model(trainset, devset, device, n_epochs=50):  #executa o ciclo de treinamento do modelo, ajustando gradualmente os parâmetros do modelo com base nas previsões e calculando a perda. 
                #O agendamento da taxa de aprendizado ajuda a controlar a taxa de convergência durante o treinamento. ORIGINAL 200
    dataloader = torch.utils.data.DataLoader(trainset, pin_memory=(device=='cuda'), num_workers=0, collate_fn=EMGDataset.collate_raw, batch_sampler=SizeAwareSampler(trainset, 128000)) #ORIGINAL 128000
        #^ configura o DataLoader para fornecer os dados de treinamento em lotes, com opções adicionais de otimização para dispositivos CUDA.

    n_chars = len(devset.text_transform.chars)  # Calcula o número de caracteres únicos no conjunto de dados de validação (devset). O atributo text_transform.chars representa a lista de caracteres nos dados de texto. 
    model = SpeechTransformer(num_classes=n_chars+1, d_model=144, num_heads=4, input_dim=devset.num_features).to(device)
   #devset.num_features representa o número de recursos (ou características) de entrada do modelo, enquanto n_chars+1 representa o número de classes de saída
                    #do modelo. Nesse caso, o número de classes de saída (opções de fonemas) é incrementado em 1 porque é adicionado um símbolo de espaço em branco (blank symbol) adicional. 

    if FLAGS.start_training_from is not None:
        state_dict = torch.load(FLAGS.start_training_from)
        model.load_state_dict(state_dict, strict=False)
        #^se houver um modelo pré-treinado fornecido para continuar o treinamento, o estado do modelo é carregado a partir do arquivo especificado por FLAGS.start_training_from usando a função torch.load(). 
        # Em seguida, o estado do modelo é carregado no modelo atual. O argumento strict=False permite que os pesos sejam carregados mesmo que a estrutura do modelo não seja idêntica, o que é útil para carregar 
        # partes de um modelo pré-treinado em um modelo com arquitetura ligeiramente modificada.

    optim = torch.optim.AdamW(model.parameters(), lr=FLAGS.learning_rate, weight_decay=FLAGS.l2) #O otimizador é configurado usando o algoritmo AdamW. Ele otimiza os parâmetros do modelo durante o treinamento. 
    #Os parâmetros model.parameters() fornecem a lista de parâmetros do modelo que serão otimizados. O argumento lr=FLAGS.learning_rate define a taxa de aprendizado inicial para o otimizador. O argumento 
    # weight_decay=FLAGS.l2 configura o termo de decaimento de peso (weight decay), que controla a regularização L2 aplicada aos parâmetros do modelo durante o treinamento.
    lr_sched = torch.optim.lr_scheduler.MultiStepLR(optim, milestones=[125,150,175], gamma=.5) #O agendador de taxa de aprendizado é configurado usando o torch.optim.lr_scheduler.MultiStepLR. Ele ajusta a taxa
    #de aprendizado ao longo do treinamento, reduzindo-a em certos marcos (milestones) predefinidos. Os marcos são definidos por milestones=[125, 150, 175], o que significa que a taxa de aprendizado será reduzida 
    # pela metade nessas épocas específicas. O argumento gamma=.5 define o fator de redução da taxa de aprendizado. Portanto, a cada marco especificado, a taxa de aprendizado é multiplicada por gamma, reduzindo-a 
    # pela metade. Isso é útil para controlar a taxa de aprendizado ao longo do treinamento, permitindo ajustes finos à medida que o treinamento progride.

    def set_lr(new_lr):#atualiza a taxa de aprendizado do otimizador. Ela itera sobre os grupos de parâmetros do otimizador (optim.param_groups) e define o valor da taxa de aprendizado ('lr') para new_lr. 
        #Essa função é usada para definir manualmente a taxa de aprendizado em momentos específicos durante o treinamento.
        for param_group in optim.param_groups:
            param_group['lr'] = new_lr

    target_lr = FLAGS.learning_rate 
    def schedule_lr(iteration):  # realiza um aquecimento linear da taxa de aprendizado, aumentando-a gradualmente até atingir o valor desejado durante as iterações de aquecimento. Após o término do aquecimento 
        #linear, a taxa de aprendizado será controlada pelo agendador de taxa de aprendizado definido anteriormente.
        iteration = iteration + 1
        if iteration <= FLAGS.learning_rate_warmup:
            set_lr(iteration*target_lr/FLAGS.learning_rate_warmup)
            

    batch_idx = 0
    optim.zero_grad()
    for epoch_idx in range(n_epochs):
        losses = []
        total_loss = 0.0
        for example in dataloader:
            schedule_lr(batch_idx) #Para cada lote de exemplos no dataloader, o agendamento da taxa de aprendizado é chamado usando o índice do lote atual (schedule_lr(batch_idx)). Isso ajusta a taxa de aprendizado 
            #com base na programação definida anteriormente.

            X = combine_fixed_length(example['emg'], 200).to(device)
            X_raw = combine_fixed_length(example['raw_emg'], 200*8).to(device)
            sess = combine_fixed_length(example['session_ids'], 200).to(device)
            #^ Os dados de entrada são pré-processados para serem alimentados no modelo. Isso inclui a combinação dos comprimentos fixos dos sinais de entrada (X, X_raw e sess) usando a função combine_fixed_length,
            #  e a padronização das sequências para que tenham o mesmo comprimento usando a função nn.utils.rnn.pad_sequence.
            xraw_lenght= X_raw.size(1)
            pred = model(X_raw, xraw_lenght)
            #print('pred:', pred)
            #pred, logits = model(X, input_lengths, targets, target_lengths)
            pred = F.log_softmax(pred, 2) #As previsões são passadas pela função de ativação softmax (F.log_softmax) para obter uma distribuição de probabilidade em cada posição da sequência
            
            pred = nn.utils.rnn.pad_sequence(decollate_tensor(pred, example['lengths']), batch_first=False) # seq first, as required by ctc  
            #^ as previsões são padronizadas usando a função nn.utils.rnn.pad_sequence para ter o formato adequado necessário para calcular a perda CTC.
            y = nn.utils.rnn.pad_sequence(example['text_int'], batch_first=True).to(device) 
            loss = F.ctc_loss(pred, y, example['lengths'], example['text_int_lengths'], blank=n_chars) #A perda CTC é calculada comparando as previsões padronizadas (pred) com os rótulos de texto padronizados (y). A função 
            #F.ctc_loss é usada para calcular a perda, levando em consideração os comprimentos das sequências e o símbolo em branco (blank).
            print(loss)
            loss = loss / accumulation_steps
            losses.append(loss.item())
            loss.requires_grad_()  
            loss.backward() #Os gradientes da perda são calculados usando a função loss.backward()
            if (batch_idx+1) % accumulation_steps == 0: #A otimização dos parâmetros ocorre a cada 2 lotes (if (batch_idx+1) % 2 == 0), onde a função optim.step() atualiza os parâmetros do modelo com base nos gradientes acumulados, 
                #e optim.zero_grad() zera os gradientes para o próximo lote.
                optim.step()
                optim.zero_grad()
            batch_idx += 1
            gc.get_count()
            gc.collect()
            gc.get_count()
            
        train_loss = np.mean(losses) #A perda média é calculada como a média das perdas em cada lote durante a época atual.
        val = test(model, devset, device) #O modelo treinado é avaliado no conjunto de validação usando a função test, que retorna a taxa de erro de palavra (WER) calculada com base nas previsões do modelo.
        lr_sched.step() #O agendador de taxa de aprendizado (lr_sched) é atualizado usando o método lr_sched.step(), que ajusta a taxa de aprendizado com base no número atual de épocas.
        logging.info(f'finished epoch {epoch_idx+1}/{n_epochs} - training loss: {train_loss:.4f} validation WER: {val*100:.2f}')
        print(f'finished epoch {epoch_idx+1} - training loss: {train_loss:.4f} validation WER: {val*100:.2f}')
        torch.save(model.state_dict(), os.path.join(FLAGS.output_directory,'model.pt')) 
        #Salva os parâmetros do modelo no disco.
        #torch.cuda.empty_cache()

    model.load_state_dict(torch.load(os.path.join(FLAGS.output_directory,'model.pt'))) # re-load best parameters  #Recarrega os melhores parâmetros do modelo com base no arquivo salvo.
    return model

def evaluate_saved(): #avalia um modelo previamente treinado em um conjunto de teste
    device = 'cuda' if torch.cuda.is_available() and not FLAGS.debug else 'cpu'
    testset = EMGDataset(test=True) #Cria uma instância do conjunto de dados de teste (testset) usando a classe EMGDataset, indicando que é para o conjunto de teste, por meio do argumento test=True.
    n_chars = len(testset.text_transform.chars) #Obtém o número de classes de saída do modelo, que é igual ao comprimento do alfabeto usado na transformação de texto (text_transform) do conjunto de teste.
    model = Model(testset.num_features, n_chars+1).to(device) #Cria uma instância do modelo (model) usando a classe Model, especificando o número de recursos de entrada (testset.num_features) e o número de classes 
    #de saída (n_chars+1). O modelo é movido para o dispositivo especificado.
    model.load_state_dict(torch.load(FLAGS.evaluate_saved)) #Carrega os parâmetros salvos do modelo a partir do arquivo indicado pela flag
    print('WER:', test(model, testset, device)) #Realiza a avaliação do modelo carregado no conjunto de teste usando a função test(), e imprime a taxa de erro de palavra (WER) resultante.

def main():
    os.makedirs(FLAGS.output_directory, exist_ok=True) # Cria o diretório de saída especificado pela sinalização FLAGS.output_directory se ele não existir. A sinalização exist_ok=True permite que o diretório 
    #seja criado mesmo se ele já existir.
    logging.basicConfig(handlers=[ #Configura o sistema de registro (logging) para gravar mensagens em um arquivo de log e exibi-las no console.
            logging.FileHandler(os.path.join(FLAGS.output_directory, 'log.txt'), 'w'),
            logging.StreamHandler()
            ], level=logging.INFO, format="%(message)s")

    logging.info(subprocess.run(['git','rev-parse','HEAD'], stdout=subprocess.PIPE, universal_newlines=True).stdout)
    logging.info(subprocess.run(['git','diff'], stdout=subprocess.PIPE, universal_newlines=True).stdout)

    logging.info(sys.argv)

    dataset = EMGDataset(dev=False,test=False) #Cria uma instância do conjunto de dados de treinamento (trainset) usando a classe EMGDataset, indicando que não é para usar o conjunto de desenvolvimento 
    #(dev=False) e o conjunto de teste (test=False).

    fraction = 0.1  # Fração dos exemplos desejados
    #trainset = dataset
    trainset = dataset.subset(fraction)

    devset = EMGDataset(dev=True) # Cria uma instância do conjunto de dados de desenvolvimento (devset) usando a classe EMGDataset, indicando que é para usar o conjunto de desenvolvimento (dev=True).
    logging.info('output example: %s', devset.example_indices[0])
    logging.info('train / dev split: %d %d',len(trainset),len(devset))

    device = 'cuda' if torch.cuda.is_available() and not FLAGS.debug else 'cpu'

    model = train_model(trainset, devset, device)

if __name__ == '__main__':
    if FLAGS.evaluate_saved is not None:
        evaluate_saved()
    else:
        main()